In [1]:
import requests
import json
import pandas as pd
from datetime import datetime

# -----------------------------
# 1) TVÉ ÚDAJE Z NETATMO DEV PORTÁLU
# -----------------------------
CLIENT_ID = "6975d41d173e20a68d0a6585"
CLIENT_SECRET = "SDTuaVw1ZMC8xhJ8vlozLqx1a"
USERNAME = "marek.dejdar@gmail.com"
PASSWORD = "m.Dejdam1"

# -----------------------------
# 2) FUNKCE PRO ZÍSKÁNÍ TOKENU
# -----------------------------
def get_token():
    url = "https://api.netatmo.com/oauth2/token"
    payload = {
        "grant_type": "password",
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        "username": USERNAME,
        "password": PASSWORD,
        "scope": "read_station read_thermostat"
    }
    r = requests.post(url, data=payload)
    r.raise_for_status()
    token = r.json()["access_token"]
    return token

# -----------------------------
# 3) FUNKCE PRO STAŽENÍ DAT
# -----------------------------
def get_station_data(token):
    url = "https://api.netatmo.com/api/getstationsdata"
    headers = {"Authorization": f"Bearer {token}"}
    r = requests.get(url, headers=headers)
    r.raise_for_status()
    return r.json()

# -----------------------------
# 4) STAŽENÍ DAT
# -----------------------------
token = get_token()
data = get_station_data(token)

# Zobrazení struktury
data


HTTPError: 403 Client Error: Forbidden for url: https://api.netatmo.com/oauth2/token

In [ ]:
# Extrakce měření z hlavního modulu
devices = data["body"]["devices"]

rows = []

for dev in devices:
    module_name = dev["module_name"]
    dashboard = dev.get("dashboard_data", {})

    if "Temperature" in dashboard:
        rows.append({
            "module": module_name,
            "time": datetime.fromtimestamp(dashboard["time_utc"]),
            "temperature": dashboard["Temperature"],
            "humidity": dashboard.get("Humidity"),
            "co2": dashboard.get("CO2")
        })

    # Podřízené moduly
    for mod in dev.get("modules", []):
        module_name = mod["module_name"]
        dashboard = mod.get("dashboard_data", {})

        if "Temperature" in dashboard:
            rows.append({
                "module": module_name,
                "time": datetime.fromtimestamp(dashboard["time_utc"]),
                "temperature": dashboard["Temperature"],
                "humidity": dashboard.get("Humidity"),
                "co2": dashboard.get("CO2")
            })

df = pd.DataFrame(rows)
df


In [ ]:
df.to_csv("netatmo_log.csv", index=False)
